In [1]:
import pandas as pd
import pathlib
import ast
import os
import shutil

In [2]:
# Check the directory 
os.getcwd()

'/Users/tylerryoo/PycharmProjects/luxembourg_crawling_hass'

## (1) Move the extracted files to corresponding folders --> data_[date]

### ACTION: CHANGE THE DATE

In [3]:
date = "01_23_2023"
week = 'week_8'

In [4]:
# new_dir_name = input('Directory name: ')
new_dir_name = 'data_' + str(date)
new_dir = pathlib.Path('/Users/tylerryoo/PycharmProjects/luxembourg_crawling_hass', new_dir_name)
new_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
file_names = ['/athome_links_final_' + str(date) + '.csv',
              '/athome_data_final_' + str(date) + '.csv',
              '/athome_expired_final_' + str(date) + '.csv'
             ]

In [ ]:
for file_name in file_names:

    filename = file_name

    original = (r'/Users/tylerryoo/PycharmProjects/luxembourg_crawling_hass' + filename)
    target = (r'/Users/tylerryoo/PycharmProjects/luxembourg_crawling_hass/data_' + str(date) + filename)

    shutil.move(original, target)

## (2) Merging Base Data & New Data

### 1. Read the base_data

In [5]:
os.getcwd()

'/Users/tylerryoo/PycharmProjects/luxembourg_crawling_hass'

In [6]:
base_data = pd.read_csv('base_database.csv', index_col = 0)

In [7]:
base_data.sample(3)

,property,price,location,proj_link,link,agency,detail,week_1,week_2,week_3,week_4,week_5,week_6,week_6.5,week_6.7,week_7,week_7.2,week_7.5
2737,Apartment 2 bedrooms in Luxembourg-Belair,"967,927 €",Luxembourg-Belair,https://www.athome.lu/en/buy/apartment/luxembo...,https://www.athome.lu/en/buy/apartment/luxembo...,CALTEUX SERGE IMMOBILIER,"{'Sale price': '967,927 €', 'Livable surface':...",newly_added_200,already_collected_200,already_collected_200,already_collected_200,already_collected_200,link_expired_410,link_expired_410,link_expired_410,link_expired_410,link_expired_410,link_expired_410
5716,House 2 bedrooms in Lintgen,"829,000 €",Luxembourg-Neudorf,https://www.athome.lu/en/buy/house/lintgen/id-...,https://www.athome.lu/en/buy/house/lintgen/id-...,The Real Est8 SA,"{'Sale price': '829,000 €', 'Availability': 'I...",newly_added_200,already_collected_200,already_collected_200,already_collected_200,already_collected_200,already_collected_200,already_collected_200,already_collected_200,already_collected_200,already_collected_200,already_collected_200
7794,Apartment 2 bedrooms in Luxembourg-Belair,"1,619,872 €",Luxembourg-Belair,https://www.athome.lu/en/buy/apartment/luxembo...,https://www.athome.lu/en/buy/apartment/luxembo...,Property SA,"{'Sale price': '1,619,872 €', 'Availability': ...",newly_added_200,already_collected_200,already_collected_200,already_collected_200,already_collected_200,already_collected_200,already_collected_200,already_collected_200,already_collected_200,already_collected_200,already_collected_200


### 2. Read the added_data

In [8]:
data_filename = "data_" + date + "/athome_data_final_" + date + ".csv"

In [9]:
links_filename = "data_" + date + "/athome_links_final_" + date + ".csv"

In [10]:
expired_filename = "data_" + date + "/athome_expired_final_" + date + ".csv"

In [11]:
added_data = pd.read_csv(data_filename, index_col = 0).drop_duplicates(subset = 'link')

In [12]:
added_links = pd.read_csv(links_filename, index_col = 0).drop_duplicates()

In [13]:
added_expired = pd.read_csv(expired_filename, index_col = 0).drop_duplicates()

In [14]:
print("Length of added_data: ", len(added_data), len(added_links), len(added_expired))

Length of added_data:  9898 9406 9898


### 2. Merging Added Data

In [15]:
added_data = added_data.merge(added_links, how = 'left', on = ['proj_link', 'location'])

In [16]:
added_data = added_data.merge(added_expired, how = 'left', on = 'link')

In [17]:
added_data['status'] = added_data['collect_status'] +"_"+ added_data["expire_status"].astype(str)

In [18]:
added_data.loc[added_data.expire_status == 410, ['status']] = 'link_expired_410'

In [19]:
added_data.loc[added_data.expire_status == 404, ['status']] = 'link_expired_404'

In [20]:
added_data.expire_status.unique()

array([200])

In [21]:
added_data.status.unique()

array(['already_collected_200', 'newly_added_200'], dtype=object)

In [22]:
print("Length of base_data and added_data: ", len(base_data), len(added_data))

Length of base_data and added_data:  19703 9898


In [23]:
# > FIRST ITERATION: WEEK1 & WEEK2 < 
# Remove collect_status and expire_status when merging week1 and week2
# base_data = base_data.drop(['collect_status', 'expire_status'], axis = 1)

In [24]:
# > AFTER FIRST ITERATION <
added_data = added_data.drop(['collect_status', 'expire_status'], axis = 1)

In [25]:
added_data.head(3)

,property,price,location,proj_link,link,agency,detail,status
0,Apartment 1 bedroom in Bissen,"621,556 €",Bissen,https://www.athome.lu/en/buy/new-property/apar...,http://athome.lu/en/buy/apartment/bissen/id-76...,Choices Real Estate,"{'Sale price': '621,556 €', 'Availability': 'T...",already_collected_200
1,Apartment 1 bedroom in Bissen,"637,122 €",Bissen,https://www.athome.lu/en/buy/new-property/apar...,http://athome.lu/en/buy/apartment/bissen/id-76...,Choices Real Estate,"{'Sale price': '637,122 €', 'Availability': 'T...",already_collected_200
2,Apartment 4 bedrooms in Bissen,"936,500 €",Bissen,https://www.athome.lu/en/buy/new-property/apar...,http://athome.lu/en/buy/apartment/bissen/id-76...,Choices Real Estate,"{'Sale price': '936,500 €', 'Availability': 'T...",already_collected_200


In [26]:
added_data[added_data.status == 'link_expired_410']

,property,price,location,proj_link,link,agency,detail,status


### 3. Merging Base Data & Added Data ==> New Data

In [27]:
new_data = base_data.merge(added_data, how = 'outer', on = ['property','price', 'location', 'proj_link',
                                                                 'link', 'agency', 'detail'], indicator = True)

In [28]:
# new_data = new_data.drop(week, axis = 1)

In [29]:
# # Handle duplicates: when merging week 1 & week 2
# new_data = new_data.loc[(new_data['_merge'] =='both') | ~(new_data['link'].duplicated() & (new_data['status_y'] == 'already_collected_200'))]

In [30]:
# Handle duplicates but only deleting ones where the first status entry is Null
new_data = new_data.loc[(new_data['_merge'] =='both') | ~(new_data['link'].duplicated() & (new_data['status'] == 'already_collected_200'))]

In [31]:
new_data = new_data.drop_duplicates(subset = ['property', 'price', 'location', 'proj_link', 'agency'])

In [32]:
print("Length of New Data after removing duplicates: ", len(new_data))

Length of New Data after removing duplicates:  20071


In [33]:
# Drop the _merge column as it is no longer needed 
new_data = new_data.drop(['_merge'], axis = 1)

In [34]:
# Recently added status_col
status_col = new_data.columns[-1]

In [35]:
status_col

'status'

In [36]:
# Fill in empty values of the recently added status column (i.e. status_y) with already_collected_200
# --> this was empty because these rows were part of a project 
new_data.loc[new_data[status_col].isnull(), [status_col]] = 'already_collected_200'

In [37]:
# # ONLY FIRST TIME RUNNING USING ORIGINAL DATA
# new_data = new_data.rename(columns = {'status_x': 'week_1', 'status_y': 'week_2'})

In [38]:
# > AFTER FIRST ITERATION <
new_data = new_data.rename(columns = {'status': week})

In [39]:
new_data.head(3)

,property,price,location,proj_link,link,agency,detail,week_1,week_2,week_3,week_4,week_5,week_6,week_6.5,week_6.7,week_7,week_7.2,week_7.5,week_8
0,Apartment 2 bedrooms in Oberkorn,"888,535 €",Oberkorn,https://www.athome.lu/en/buy/new-property/apar...,http://athome.lu/en/buy/apartment/oberkorn/id-...,AFIL IMMO S.A.,"{'Sale price': '888,535 €', 'Availability': '0...",newly_added_200,already_collected_200,already_collected_200,already_collected_200,already_collected_200,already_collected_200,link_expired_410,link_expired_410,link_expired_410,link_expired_410,link_expired_410,already_collected_200
1,Apartment 2 bedrooms in Oberkorn,"893,736 €",Oberkorn,https://www.athome.lu/en/buy/new-property/apar...,http://athome.lu/en/buy/apartment/oberkorn/id-...,AFIL IMMO S.A.,"{'Sale price': '893,736 €', 'Availability': '0...",newly_added_200,already_collected_200,already_collected_200,already_collected_200,already_collected_200,already_collected_200,link_expired_410,link_expired_410,link_expired_410,link_expired_410,link_expired_410,already_collected_200
2,Apartment 1 bedroom in Oberkorn,"555,638 €",Oberkorn,https://www.athome.lu/en/buy/new-property/apar...,http://athome.lu/en/buy/apartment/oberkorn/id-...,AFIL IMMO S.A.,"{'Sale price': '555,638 €', 'Availability': '0...",newly_added_200,already_collected_200,already_collected_200,already_collected_200,already_collected_200,already_collected_200,link_expired_410,link_expired_410,link_expired_410,link_expired_410,link_expired_410,already_collected_200


In [40]:
# ONLY FIRST TIME RUNNING USING ORIGINAL DATA
# new_data = new_data.drop(['collect_status', 'expire_status'], axis = 1)

In [41]:
new_data[new_data[week] == 'link_expired_200']

,property,price,location,proj_link,link,agency,detail,week_1,week_2,week_3,week_4,week_5,week_6,week_6.5,week_6.7,week_7,week_7.2,week_7.5,week_8


In [42]:
new_data.head(3)

,property,price,location,proj_link,link,agency,detail,week_1,week_2,week_3,week_4,week_5,week_6,week_6.5,week_6.7,week_7,week_7.2,week_7.5,week_8
0,Apartment 2 bedrooms in Oberkorn,"888,535 €",Oberkorn,https://www.athome.lu/en/buy/new-property/apar...,http://athome.lu/en/buy/apartment/oberkorn/id-...,AFIL IMMO S.A.,"{'Sale price': '888,535 €', 'Availability': '0...",newly_added_200,already_collected_200,already_collected_200,already_collected_200,already_collected_200,already_collected_200,link_expired_410,link_expired_410,link_expired_410,link_expired_410,link_expired_410,already_collected_200
1,Apartment 2 bedrooms in Oberkorn,"893,736 €",Oberkorn,https://www.athome.lu/en/buy/new-property/apar...,http://athome.lu/en/buy/apartment/oberkorn/id-...,AFIL IMMO S.A.,"{'Sale price': '893,736 €', 'Availability': '0...",newly_added_200,already_collected_200,already_collected_200,already_collected_200,already_collected_200,already_collected_200,link_expired_410,link_expired_410,link_expired_410,link_expired_410,link_expired_410,already_collected_200
2,Apartment 1 bedroom in Oberkorn,"555,638 €",Oberkorn,https://www.athome.lu/en/buy/new-property/apar...,http://athome.lu/en/buy/apartment/oberkorn/id-...,AFIL IMMO S.A.,"{'Sale price': '555,638 €', 'Availability': '0...",newly_added_200,already_collected_200,already_collected_200,already_collected_200,already_collected_200,already_collected_200,link_expired_410,link_expired_410,link_expired_410,link_expired_410,link_expired_410,already_collected_200


In [43]:
# from bs4 import BeautifulSoup
# from lxml import etree
# import requests
# import random
# import time

In [44]:
# link_index = 0
# status_check = []
# window = 6.1
# chrome = random.randint(0,105)
# chrome_2 = random.randint(149, 5000)

# for link, status in zip(new_data.link, new_data[week]):
#     headers = {'User-Agent': 'Mozilla/5.0 (Windows NT ' + str(window) + '; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/' + str(chrome) + '.0.3626.' + str(chrome_2) + ' Safari/537.36',
#                'Content-Type': 'text/html'}
#     if link_index % 100 == 0:
#         print()
#         print(headers)
#         print()
#         window += 0.1
#         window = round(window, 2)
#         chrome = random.randint(0,105)
#         chrome_2 = random.randint(149, 5000)

#     # locate link index
#     print("Link Index: ", link_index, "|", link)
#     link_index += 1
#     print(link)
#     webpage = requests.get(link, headers = headers)
#     time.sleep(0.05)
#     soup = BeautifulSoup(webpage.content, 'html.parser')
#     dom = etree.HTML(str(soup))

#     # keep track of pages that expired
#     if webpage.status_code == 404 or webpage.status_code == 410:
#         if status == 'link_expired_404':
#             status_check.append(status)
#         elif status == 'link_expired_410':
#             status_check.append(status)
#         else:
#             status_check.append('link_expired_' + str(webpage.status_code))

#     # if page not expired
#     elif webpage.status_code == 200:
#         status_check.append(status)

In [45]:
# Resetting the kernel: nbstripout Summary.ipynb

In [46]:
# https://superfastpython.com/threadpoolexecutor-website-status/
# example of pinging the status of a set of websites asynchronously
from urllib.request import urlopen
from urllib.error import URLError
from urllib.error import HTTPError
from http import HTTPStatus
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed
import random
link_check = []
status_check = []

window = 6.1
 
# get the status of a website
def get_website_status(url):
    # handle connection errors
    try:
        # open a connection to the server with a timeout
        with urlopen(url, timeout=10) as connection:
            # get the response code, e.g. 200
            code = connection.getcode()
            if code == 200:
                code = 200
                return code
    except HTTPError as e:
        if e.code == 410:
            return 410
        elif e.code == 502:
            return 502
        elif e.code == 404:
            return 404
#         else:
#             print(e.code, 'UNUSUAL ERROR')
#     except URLError as e:
#         print(e.reason)
#         return e.reason
    except Exception as e:
        i = 0
        while i < 100:
            try:
                i += 1
                chrome = random.randint(0,105)
                chrome_2 = random.randint(149, 5000)
                headers = {'User-Agent': 'Mozilla/5.0 (Windows NT ' + str(window) + '; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/' + str(chrome) + '.0.3626.' + str(chrome_2) + ' Safari/537.36',
                   'Content-Type': 'text/html'}
                webpage = requests.get(url, headers = headers, timeout = 10)
                i = 100
                print(i, webpage.status_code)
                return webpage.status_code   
            except:
                continue 
        print(e)
        return e
 
# interpret an HTTP response code into a status
def get_status(code):
    if code == HTTPStatus.OK:
        return 'OK'
    return 'ERROR'
 
# check status of a list of websites
def check_status_urls(urls):
    # create the thread pool
    with ThreadPoolExecutor(len(urls)) as executor:
        # check the status of each url
        results = executor.map(get_website_status, urls)
        # enumerate the results in the same order as the list of urls
        for i,code in enumerate(results):
            # get the url
            url = urls[i]
            # interpret the status
            status = get_status(code)
            # report status
            link_check.append(url)
            status_check.append(code)
            print(f'{url:20s}\t{status:5s}\t{code}')
 
# list of urls to check
URLS = new_data.link.to_list()
# check all urls
check_status_urls(URLS)

http://athome.lu/en/buy/apartment/oberkorn/id-7688327.html	ERROR	410
http://athome.lu/en/buy/apartment/oberkorn/id-7688329.html	ERROR	410
http://athome.lu/en/buy/apartment/oberkorn/id-7688334.html	ERROR	410
http://athome.lu/en/buy/commercial-property/oberkorn/id-7688330.html	ERROR	410
http://athome.lu/en/buy/apartment/oberkorn/id-7688332.html	ERROR	410
http://athome.lu/en/buy/apartment/oberkorn/id-7688335.html	ERROR	410
http://athome.lu/en/buy/apartment/oberkorn/id-7688331.html	ERROR	410
http://athome.lu/en/buy/apartment/oberkorn/id-7688333.html	ERROR	410
http://athome.lu/en/buy/apartment/oberkorn/id-7688341.html	ERROR	410
http://athome.lu/en/buy/apartment/oberkorn/id-7688342.html	ERROR	410
http://athome.lu/en/buy/apartment/oberkorn/id-7688338.html	ERROR	410
http://athome.lu/en/buy/apartment/oberkorn/id-7688348.html	ERROR	410
http://athome.lu/en/buy/apartment/oberkorn/id-7688336.html	ERROR	410
http://athome.lu/en/buy/apartment/oberkorn/id-7688339.html	ERROR	410
http://athome.lu/en/buy/

https://www.athome.lu/en/buy/house/rodange/id-7344705.html	OK   	200
https://www.athome.lu/en/buy/house/lamadelaine/id-7595823.html	OK   	200
https://www.athome.lu/en/buy/house/luxembourg/id-7637368.html	OK   	200
https://www.athome.lu/en/buy/apartment/bereldange/id-7716428.html	OK   	200
https://www.athome.lu/en/buy/apartment/luxembourg/id-7715140.html	OK   	200
https://www.athome.lu/en/buy/apartment/medernach/id-7734721.html	ERROR	410
https://www.athome.lu/en/buy/house/dudelange/id-7724270.html	ERROR	410
https://www.athome.lu/en/buy/apartment/bertrange/id-7725009.html	ERROR	410
https://www.athome.lu/en/buy/apartment/eischen/id-7493479.html	OK   	200
https://www.athome.lu/en/buy/house/tuntange/id-7683362.html	OK   	200
https://www.athome.lu/en/buy/apartment/kehlen/id-7680643.html	OK   	200
https://www.athome.lu/en/buy/apartment/kehlen/id-7680653.html	OK   	200
https://www.athome.lu/en/buy/apartment/kehlen/id-7680664.html	OK   	200
https://www.athome.lu/en/buy/apartment/kehlen/id-76806

http://athome.lu/en/buy/apartment/moestroff/id-7593995.html	OK   	200
http://athome.lu/en/buy/apartment/moestroff/id-7593996.html	OK   	200
http://athome.lu/en/buy/apartment/moestroff/id-7593997.html	OK   	200
http://athome.lu/en/buy/apartment/moestroff/id-7593998.html	OK   	200
http://athome.lu/en/buy/apartment/moestroff/id-7593999.html	OK   	200
http://athome.lu/en/buy/apartment/luxembourg/id-7534673.html	OK   	200
http://athome.lu/en/buy/apartment/luxembourg/id-7534674.html	OK   	200
http://athome.lu/en/buy/apartment/luxembourg/id-7534715.html	OK   	200
http://athome.lu/en/buy/apartment/luxembourg/id-7534753.html	OK   	200
http://athome.lu/en/buy/apartment/luxembourg/id-7534877.html	OK   	200
http://athome.lu/en/buy/apartment/luxembourg/id-7534924.html	OK   	200
http://athome.lu/en/buy/apartment/luxembourg/id-7534944.html	OK   	200
http://athome.lu/en/buy/apartment/luxembourg/id-7534955.html	OK   	200
http://athome.lu/en/buy/apartment/luxembourg/id-7534957.html	OK   	200
http://atho

https://www.athome.lu/en/buy/apartment/esch-sur-alzette/id-7817669.html	OK   	200
https://www.athome.lu/en/buy/house/oberkorn/id-7817662.html	OK   	200
https://www.athome.lu/en/buy/apartment/mamer/id-7817655.html	OK   	200
https://www.athome.lu/en/buy/apartment/luxembourg/id-7817654.html	OK   	200
https://www.athome.lu/en/buy/apartment/esch-sur-alzette/id-7817653.html	OK   	200
https://www.athome.lu/en/buy/apartment/esch-sur-alzette/id-7817652.html	OK   	200
https://www.athome.lu/en/buy/apartment/differdange/id-7817651.html	OK   	200
https://www.athome.lu/en/buy/apartment/kehlen/id-7817643.html	OK   	200
https://www.athome.lu/en/buy/house/dippach/id-7817642.html	OK   	200
https://www.athome.lu/en/buy/apartment/schifflange/id-7817627.html	OK   	200
https://www.athome.lu/en/buy/house/dudelange/id-7816839.html	OK   	200
https://www.athome.lu/en/buy/house/dudelange/id-7816837.html	OK   	200
https://www.athome.lu/en/buy/house/mamer/id-7817626.html	OK   	200
https://www.athome.lu/en/buy/hous

### IF Nothing is Listed Below, Proceed

In [47]:
new_status = []

for i, j in zip(new_data[week], status_check):
    if i.split('_')[2] == str(j): # if matching status, no changes needed
        if i == 'link_expired_200':
            print(i,j)
        new_status.append(i)
    else:
        if str(j) == '200':
            new_status.append('newly_added_200')
        elif len(str(j)) > 3:
            print('unusual error message --> ',i, j)
            
            new_status.append('link_expired_ERROR')
        else:
            new_status.append('link_expired_' + str(j))

In [48]:
new_data = new_data.drop(week, axis = 1)

In [49]:
new_data[week] = new_status

In [50]:
# new_data[week] = new_data[week].replace(['link_expired410'], 'link_expired_410')

In [51]:
for i in new_data[new_data[week] == 'link_expired_ERROR'].link:
    print(i)

In [52]:
new_data.groupby(week)[week].count()

week_8
already_collected_200    13412
link_expired_410          6291
newly_added_200            368
Name: week_8, dtype: int64

In [53]:
new_data[week].unique()

array(['link_expired_410', 'already_collected_200', 'newly_added_200'],
      dtype=object)

In [54]:
new_data.head(5)

,property,price,location,proj_link,link,agency,detail,week_1,week_2,week_3,week_4,week_5,week_6,week_6.5,week_6.7,week_7,week_7.2,week_7.5,week_8
0,Apartment 2 bedrooms in Oberkorn,"888,535 €",Oberkorn,https://www.athome.lu/en/buy/new-property/apar...,http://athome.lu/en/buy/apartment/oberkorn/id-...,AFIL IMMO S.A.,"{'Sale price': '888,535 €', 'Availability': '0...",newly_added_200,already_collected_200,already_collected_200,already_collected_200,already_collected_200,already_collected_200,link_expired_410,link_expired_410,link_expired_410,link_expired_410,link_expired_410,link_expired_410
1,Apartment 2 bedrooms in Oberkorn,"893,736 €",Oberkorn,https://www.athome.lu/en/buy/new-property/apar...,http://athome.lu/en/buy/apartment/oberkorn/id-...,AFIL IMMO S.A.,"{'Sale price': '893,736 €', 'Availability': '0...",newly_added_200,already_collected_200,already_collected_200,already_collected_200,already_collected_200,already_collected_200,link_expired_410,link_expired_410,link_expired_410,link_expired_410,link_expired_410,link_expired_410
2,Apartment 1 bedroom in Oberkorn,"555,638 €",Oberkorn,https://www.athome.lu/en/buy/new-property/apar...,http://athome.lu/en/buy/apartment/oberkorn/id-...,AFIL IMMO S.A.,"{'Sale price': '555,638 €', 'Availability': '0...",newly_added_200,already_collected_200,already_collected_200,already_collected_200,already_collected_200,already_collected_200,link_expired_410,link_expired_410,link_expired_410,link_expired_410,link_expired_410,link_expired_410
3,Retail in Oberkorn,"179,384 €",Oberkorn,https://www.athome.lu/en/buy/new-property/apar...,http://athome.lu/en/buy/commercial-property/ob...,AFIL IMMO S.A.,"{'Sale price': '179,384 €', 'Availability': '0...",newly_added_200,already_collected_200,already_collected_200,already_collected_200,already_collected_200,already_collected_200,link_expired_410,link_expired_410,link_expired_410,link_expired_410,link_expired_410,link_expired_410
4,Apartment 1 bedroom in Oberkorn,"581,196 €",Oberkorn,https://www.athome.lu/en/buy/new-property/apar...,http://athome.lu/en/buy/apartment/oberkorn/id-...,AFIL IMMO S.A.,"{'Sale price': '581,196 €', 'Availability': '0...",newly_added_200,already_collected_200,already_collected_200,already_collected_200,already_collected_200,already_collected_200,link_expired_410,link_expired_410,link_expired_410,link_expired_410,link_expired_410,link_expired_410


In [55]:
new_data.to_csv('base_database.csv', encoding = 'utf-8-sig')
new_data.to_csv('base_database_copy.csv', encoding = 'utf-8-sig')

## (3) Add property type + Split Detail into Separate Columns 

In [56]:
# original_data = original_data[['property', 'price', 'location', 'proj_link', 'link', 'agency', 'detail', 'status']]

In [57]:
value = new_data['property'].str.split(' ').str[:1].apply(lambda a: a[0] if pd.notnull(a) else None)

new_data.insert(1,'prop_type',value)

In [58]:
new_data['price'] = new_data['price'].str.replace(r'€', '')

In [59]:
new_data['price'] = new_data['price'].str.lstrip('From ')

In [60]:
new_data['price'] = new_data['price'].str.replace(r',', '')

In [61]:
new_data['price'] = new_data['price'].replace({'Marked as sold':99999999})

In [62]:
new_data.head(1)

,property,prop_type,price,location,proj_link,link,agency,detail,week_1,week_2,week_3,week_4,week_5,week_6,week_6.5,week_6.7,week_7,week_7.2,week_7.5,week_8
0,Apartment 2 bedrooms in Oberkorn,Apartment,888535,Oberkorn,https://www.athome.lu/en/buy/new-property/apar...,http://athome.lu/en/buy/apartment/oberkorn/id-...,AFIL IMMO S.A.,"{'Sale price': '888,535 €', 'Availability': '0...",newly_added_200,already_collected_200,already_collected_200,already_collected_200,already_collected_200,already_collected_200,link_expired_410,link_expired_410,link_expired_410,link_expired_410,link_expired_410,link_expired_410


In [63]:
new_info = []
for i in new_data['detail']:
    if i == "{}":
        new_info.append({})
    elif pd.isnull(i):
        new_info.append({})
    else:
        new_info.append(ast.literal_eval(i))

In [64]:
new_data['detail'] = new_info

In [65]:
new_data['detail'].apply(pd.Series, dtype = 'str').head(1)

,Sale price,Availability,Livable surface,Lift,Number of bedrooms,Renovation year,Living room,Fitted kitchen,Separate kitchen,Shower rooms,...,Total floors,Indoor parking space(s),Rent,Security deposit,Furnished,Type of mandate,Life annuity sale,Assisted living facilities,Agency commission,Number of properties within the co-ownership association
0,"888,535 €",01/12/2024,102.39 m²,Yes,2,2019,Yes,Yes,Yes,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
new_data_status = new_data.iloc[:,8:]

In [67]:
new_data_basic_info = new_data.iloc[:,:7]

In [68]:
# Reorder DataFrames: Basic Info + Details + Status
new_data = pd.concat([new_data_basic_info, new_data['detail'].apply(pd.Series, dtype = 'str'), new_data_status], axis = 1)

In [69]:
# Drop Duplicated Info: Sale Price
new_data = new_data.drop(columns = ['Sale price'])

In [70]:
new_data['Livable surface'] = new_data['Livable surface'].str.rstrip('m²')

In [71]:
new_data['Livable surface'] = pd.to_numeric(new_data['Livable surface'].str.replace(r',',''))

In [72]:
new_data.to_csv('added_data_final_' + date + '.csv', encoding = 'utf-8-sig')

### Move the final version of the file to the corresponding day or week's file

In [73]:
filename = '/added_data_final_' + str(date) + '.csv'

original = (r'/Users/tylerryoo/PycharmProjects/luxembourg_crawling_hass' + filename)
target = (r'/Users/tylerryoo/PycharmProjects/luxembourg_crawling_hass/data_' + str(date) + filename)

shutil.move(original, target)

'/Users/tylerryoo/PycharmProjects/luxembourg_crawling_hass/data_01_23_2023/added_data_final_01_23_2023.csv'

### Save Copy of Base Database

In [74]:
save_to_filename = 'data_' + date
original = (r'/Users/tylerryoo/PycharmProjects/luxembourg_crawling_hass/base_database_copy.csv')
target = (r'/Users/tylerryoo/PycharmProjects/luxembourg_crawling_hass/' + save_to_filename + '/base_database_copy.csv')

shutil.move(original, target)

'/Users/tylerryoo/PycharmProjects/luxembourg_crawling_hass/data_01_23_2023/base_database_copy.csv'

# Appendix

## (1) Move Files to original_data

In [ ]:
# import pathlib

# new_dir_name = input('Directory name: ')
# new_dir = pathlib.Path('/Users/tylerryoo/PycharmProjects/luxembourg_crawling_hass', new_dir_name)
# new_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
# file_names = ['/athome_links_final.csv',
#               '/athome_data_final.csv',
#               '/athome_expired_final.csv'
#              ]

In [ ]:
# import shutil

# for file_name in file_names:

#     filename = file_name

#     original = (r'/Users/tylerryoo/PycharmProjects/luxembourg_crawling_hass' + filename)
#     target = (r'/Users/tylerryoo/PycharmProjects/luxembourg_crawling_hass/original_data' + filename)

#     shutil.move(original, target)

## (2) Original Data Wrangling

### 1. Reading Original Files (original_data, original_links, original_expired)

In [ ]:
# # dropped duplicates based on links because multiple agencies listed identical properties 
# original_data = pd.read_csv("original_data/athome_data_final.csv", index_col = 0).drop_duplicates(subset = 'link')
# original_links = pd.read_csv("original_data/athome_links_final.csv", index_col = 0).drop_duplicates()
# original_expired = pd.read_csv("original_data/athome_expired_final.csv", index_col = 0).drop_duplicates()
# print("Length of each file: ", len(original_data), len(original_links), len(original_expired))

In [ ]:
# original_data.head(1)

In [ ]:
# original_links.head(1)

In [ ]:
# original_expired.head(1)

### 2. Merging Original Files into one and eliminating expired files

In [ ]:
# original_data = original_data.merge(original_links, how = 'left', on = ['proj_link', 'location'])
# original_data = original_data.merge(original_expired, how = 'left', on = 'link')
# original_data['status'] = original_data['collect_status'] +"_"+ original_data["expire_status"].astype(str)
# print("Length of Files after merge: ", len(original_data))

# # eliminated expired data from the original data
# original_data = original_data[~original_data['collect_status'].isnull()]

# print("Length of Files after eliminating expired files: ", len(original_data))

### 3. Save the original_data as base_data

In [ ]:
# base_data = original_data
# base_data.to_csv('base_database.csv', encoding =  'utf-8-sig')
# base_data.to_csv('base_database_copy.csv', encoding = 'utf-8-sig')

In [ ]:
# original = (r'/Users/tylerryoo/PycharmProjects/luxembourg_crawling_hass/base_database_copy.csv')
# target = (r'/Users/tylerryoo/PycharmProjects/luxembourg_crawling_hass/original_data/base_database_copy.csv')

# shutil.move(original, target)